<a href="https://colab.research.google.com/github/Mathijs995/Advanced-Econometrics-III/blob/master/AE3_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Econometrics III
## Assignment 1
The volatility index (VIX) is a measure of the volatility of the S&P 500 index, constructed from option prices; see https://www.cboe.com/micro/vix/vixwhite.pdf. The file VIX.csv contains daily data on the VIX over the years 1991–2017. We are going to work with the natural logarithm of the VIX given by $Xt = \ln(\text{VIX}_{t})$. We start by mounting Google Drive to this notebook, so we can import the data.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


It remains to download the data and apply the natural logarithm transformation.

In [0]:
import pandas as pd
import numpy as np
import scipy
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib as plt

In [0]:
# Print options
np.set_printoptions(precision=4, threshold=10000, linewidth=150, suppress=True)


def regressors_given_p(fd_X, X, p):
    """Purpose: Returns the regressor matrix mX and the regressand vector vY
    for a given number of lags p (now added a constant)
    Inputs: fd_X, X, p
    Returns: vY,mX
    """
    ones = np.ones((len(X)-p-1))
    ones = ones.reshape((ones.shape[0],1))
    vY = fd_X[p:]
    mX = np.concatenate((ones, X[p:-1], np.concatenate([fd_X[p - i - 1: - i - 1] for i in range(p)], axis=1)), axis=1)
    
#    print("fd_X=\n{}\n".format(fd_X[0:5,]))
#    print("X=\n{}\n".format(X[0:5,]))
#    print("vY =\n{}\n".format(vY[0:5,]))
#    print("mX =\n{}\n".format(mX[0:5,]))
    
    return vY, mX


def ols(vY,mX):
    """Purpose: Returns the ols estimate vector vB
    Inputs: vY,mX
    Returns: vB
    """
    vB=np.linalg.inv(mX.T@mX)@mX.T@vY
    
    return vB

def bic(vY,mX, vB):
    """Purpose: value of BIC criterion given data and parameters
    Inputs: vY,mX,vB
    Returns: BIC (scalar)
    """
    n=len(vY)
    k=len(mX.T)
    resid = vY - mX@vB
    sse = sum(resid**2)
    BIC = n*np.log(sse/n) + k*np.log(n)
    
    return BIC

def tstat(vY,mX,vB, par, h0):
    """Purpose: Returns the t-statistic for the given parameter par and h0
    Inputs: vY,mX,vB, par
    Returns: t
    """
    n=len(vY)
    k=len(mX.T)
    resid = vY - mX@vB
    sse = sum(resid**2)
    mCovhat = np.linalg.inv(mX.T@mX)*sse/(n-k) # Estimate covariance matrix
    t = (vB[par]-h0)/np.sqrt(np.diag(mCovhat)[par])
    
    return t

###########################################################
### main
def main():
    # Inputs
    data = pd.read_csv("C:/Users/Jakob/Documents/adv ectrics III/VIX.csv")
    X = np.log(data["VIX"].values)
    fd_X = np.diff(X)
    X = X.reshape((X.shape[0],1))
    fd_X = fd_X.reshape((fd_X.shape[0], 1))
    
    MAX_LAG = 50
    T_DELTA = -2.86
    

    # Initialisation
    
    plot_acf(X, lags=range(1, MAX_LAG + 1))
    plt.show()
    plot_acf(fd_X, lags=range(1, MAX_LAG + 1))
    plt.show()
    
    #Finding the optimal number of lags p
    p = 1
    for i in range(10):
        vY, mX = regressors_given_p(fd_X, X, p)
        vB = ols(vY,mX)
        BIC = bic(vY,mX, vB)
        print('BIC for ', p, ' lags is ', BIC)
        p += 1
        
    # THE OPTIMAL NUMBER OF LAGS IS 4! -> SET p=4 subsequently
    p = 4
    
    # Now do Dickey-Fuller test:
    vY, mX = regressors_given_p(fd_X, X, p)
    vB = ols(vY,mX)
    par = 1 # We are interested in the second parameter delta (first is the constant)
    h0 = 0 # We test h0: X is integrated of order one so then delta is 0
    t=tstat(vY,mX,vB, par, h0)
    print('The point estimate for delta is ', vB[par])
    print('The t-statistic for testing whether the process is integrated of ',
          'order one is', t)
# =============================================================================
# We get a high t statistic b/c even though delta hat is close to 0,
# it is very precisely estimated (n=6800). In conclusion, h0 is rejected and the 
# process is NOT integrated of order one.
# =============================================================================
    
    # Estimate spectral density
    f,Pxx  = scipy.signal.periodogram(fd_X, window='bartlett')

 

###########################################################
### start main
if __name__ == "__main__":
    main()
